In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import csv
#import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import datapane as dp
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\galci\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [3]:
# URL of page to be scraped nasa mars news
url = 'https://finances.worldbank.org/Other/IEG-World-Bank-Project-Performance-Ratings/rq9d-pctf/data'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
#print(soup.prettify())

In [4]:
# get news title, news link, news text, news date
results = soup.find_all("div", attrs={"class":"socrata-table"})

In [5]:
table = results[0].find("table")

In [6]:
body = table.find_all("tr")
head=body[0]
body_rows=body[1:]
headings=[]
for item in head.find_all("th"):
    item=(item.text).rstrip("\n")
    headings.append(item)
#print(headings)

In [7]:
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [8]:
project_df = pd.DataFrame(data=all_rows,columns=headings)

In [9]:
#print(df.columns)

In [10]:
project_df.columns=['Project ID','Project Name','Region','Country Code','Country Name','Approval Date',
       'Approval FY','Sector Board','Agreement Type','Lending Project Cost','Net Commitment',
       'Deactivation Date','Exit FY','Lending Instrument Typen','Lending Instrument','Product Line Code',
       'Product Line','IEG_EvalDate','IEG_EvalFY','IEG_EvalType','ERR at Appraisal','ERR at Completion',
       'IEG_Outcome','IEG_RDO','(disc)IEG_IDImpact','IEG_BankQualityAtEntry','IEG_BankQualityOfSupervision',
       'IEG_OverallBankPerf','(disc)IEG_BorrPrep','EG_ImplementingAgencyPerf','IEG_GovernmentPerf',
       'IEG_OverallBorrPerf','IEG_ICRQuality','(disc)IEG_Sustainability','IEG_MEQualit','IEG_SourceDocumentURL']
project_df

,Project ID,Project Name,Region,Country Code,Country Name,Approval Date,Approval FY,Sector Board,Agreement Type,Lending Project Cost,...,IEG_BankQualityOfSupervision,IEG_OverallBankPerf,(disc)IEG_BorrPrep,EG_ImplementingAgencyPerf,IEG_GovernmentPerf,IEG_OverallBorrPerf,IEG_ICRQuality,(disc)IEG_Sustainability,IEG_MEQualit,IEG_SourceDocumentURL
0,P146981,BR Piaui Productive and Social Inclu...,LCR,BR,Brazil,12/21/2015,2016,Environment,IBRD,200000000,...,SATISFACTORY,SATISFACTORY,,MODERATELY SATISFACTORY,SATISFACTORY,MODERATELY SATISFACTORY,SUBSTANTIAL,,SUBSTANTIAL,
1,P125272,NIGER - Shared Growth Credit I,AFW,NE,Niger,6/26/2012,2012,Economic Policy,IDA,50000000,...,MODERATELY UNSATISFACTORY,MODERATELY UNSATISFACTORY,,,MODERATELY UNSATISFACTORY,MODERATELY UNSATISFACTORY,HIGH,,MODEST,
2,P125288,GY - UG Science and Technology Suppo...,LCR,GY,Guyana,6/23/2011,2011,,IDA,11400000,...,MODERATELY SATISFACTORY,MODERATELY SATISFACTORY,,,,,SUBSTANTIAL,,MODEST,
3,P148099,Governance and Competitiveness DPO ...,ECA,KG,Kyrgyz Republic,7/7/2016,2017,,IDA,24000000,...,MODERATELY UNSATISFACTORY,MODERATELY UNSATISFACTORY,,NOT RATED,UNSATISFACTORY,UNSATISFACTORY,SUBSTANTIAL,,MODEST,
4,P125298,Lao PDR PRSO 8,EAP,LA,Lao People's Democratic Republic ...,8/9/2012,2013,Economic Policy,IDA,20000000,...,MODERATELY SATISFACTORY,MODERATELY UNSATISFACTORY,,,UNSATISFACTORY,UNSATISFACTORY,SUBSTANTIAL,,LOW,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,P000087,TELECOMMUNICATIONS,AFR,BJ,Benin,11/8/1988,1989,Global Information/Communications Te...,IDA,,...,SATISFACTORY,NOT RATED,SATISFACTORY,SATISFACTORY,SATISFACTORY,NOT RATED,NOT RATED,LIKELY,,
96,P000088,PUBLIC ENTERPRISES,AFR,BJ,Benin,12/16/1986,1987,Public Sector Governance,IDA,,...,SATISFACTORY,SATISFACTORY,SATISFACTORY,SATISFACTORY,SATISFACTORY,SATISFACTORY,SATISFACTORY,UNCERTAIN,,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
97,P000089,WATER II,AFR,BJ,Benin,6/30/1986,1986,Water,IDA,,...,SATISFACTORY,NOT RATED,SATISFACTORY,SATISFACTORY,UNSATISFACTORY,NOT RATED,NOT RATED,UNCERTAIN,,http://www-wds.worldbank.org/external/default/...
98,P000090,BORGOU RD.II,AFR,BJ,Benin,2/16/1988,1988,Agriculture and Rural Development ...,IDA,,...,SATISFACTORY,NOT RATED,SATISFACTORY,HIGHLY SATISFACTORY,SATISFACTORY,NOT RATED,SATISFACTORY,LIKELY,,http://www-wds.worldbank.org/external/default/...


In [11]:
project_df=project_df.loc[:,['Project ID','Project Name','Region','Country Code','Country Name','Approval Date',
       'Approval FY','Sector Board','Lending Project Cost','Net Commitment',
       'Deactivation Date','Lending Instrument','Product Line Code',
       'Product Line','IEG_Outcome','IEG_SourceDocumentURL']]
project_df

,Project ID,Project Name,Region,Country Code,Country Name,Approval Date,Approval FY,Sector Board,Lending Project Cost,Net Commitment,Deactivation Date,Lending Instrument,Product Line Code,Product Line,IEG_Outcome,IEG_SourceDocumentURL
0,P146981,BR Piaui Productive and Social Inclu...,LCR,BR,Brazil,12/21/2015,2016,Environment,200000000,200000000,8/31/2017,DPL,PE,IBRD/IDA,Moderately Satisfactory,
1,P125272,NIGER - Shared Growth Credit I,AFW,NE,Niger,6/26/2012,2012,Economic Policy,50000000,50000000,5/29/2013,DPL,PE,IBRD/IDA,Moderately Unsatisfactory,
2,P125288,GY - UG Science and Technology Suppo...,LCR,GY,Guyana,6/23/2011,2011,,11400000,9999147.78,9/30/2017,SIL,PE,IBRD/IDA,Satisfactory,
3,P148099,Governance and Competitiveness DPO ...,ECA,KG,Kyrgyz Republic,7/7/2016,2017,,24000000,24000000,9/1/2017,DPL,PE,IBRD/IDA,Moderately Unsatisfactory,
4,P125298,Lao PDR PRSO 8,EAP,LA,Lao People's Democratic Republic ...,8/9/2012,2013,Economic Policy,20000000,20000000,10/25/2012,DPL,PE,IBRD/IDA,Moderately Unsatisfactory,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,P000087,TELECOMMUNICATIONS,AFR,BJ,Benin,11/8/1988,1989,Global Information/Communications Te...,,16000000,6/30/1995,SIM,PE,IBRD/IDA,Satisfactory,
96,P000088,PUBLIC ENTERPRISES,AFR,BJ,Benin,12/16/1986,1987,Public Sector Governance,,15000000,12/31/1997,SIM,PE,IBRD/IDA,Satisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
97,P000089,WATER II,AFR,BJ,Benin,6/30/1986,1986,Water,,10000000,12/31/1993,SIL,PE,IBRD/IDA,Unsatisfactory,http://www-wds.worldbank.org/external/default/...
98,P000090,BORGOU RD.II,AFR,BJ,Benin,2/16/1988,1988,Agriculture and Rural Development ...,,20999766.12,9/30/1993,SIL,PE,IBRD/IDA,Highly Satisfactory,http://www-wds.worldbank.org/external/default/...


In [12]:
project_df.to_html('app/templates/iegData.html')

In [13]:
browser.quit()

In [14]:
gdp_df=pd.read_csv('data/gdp-per-capita-worldbank.csv')
gdp_df.head(20)

,Entity,Code,Year,"GDP per capita, PPP (constant 2011 international $)"
0,Afghanistan,AFG,2002,1063.635574
1,Afghanistan,AFG,2003,1099.194507
2,Afghanistan,AFG,2004,1062.249360
3,Afghanistan,AFG,2005,1136.123214
4,Afghanistan,AFG,2006,1161.124889
5,Afghanistan,AFG,2007,1284.775213
6,Afghanistan,AFG,2008,1298.143159
7,Afghanistan,AFG,2009,1531.173993
8,Afghanistan,AFG,2010,1614.255001
9,Afghanistan,AFG,2011,1660.739856


In [15]:
gdp_df.to_html('app/templates/gdpData.html')

In [16]:
population_df=pd.read_csv('data/world-population-by-world-regions-post-1820.csv')
population_df.head(20)

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)"
0,Afghanistan,AFG,1800,3280000
1,Afghanistan,AFG,1801,3280000
2,Afghanistan,AFG,1802,3280000
3,Afghanistan,AFG,1803,3280000
4,Afghanistan,AFG,1804,3280000
5,Afghanistan,AFG,1805,3280000
6,Afghanistan,AFG,1806,3280000
7,Afghanistan,AFG,1807,3280000
8,Afghanistan,AFG,1808,3280000
9,Afghanistan,AFG,1809,3280000


In [17]:
population_df.to_html('app/templates/populationData.html')

In [18]:
project_df=pd.read_csv('data/IEG_World_Bank_Project_Performance_Ratings.csv')

In [19]:
project_df=project_df.loc[:,['Country Name','IEG_Outcome']]
project_df.columns=['country', 'IEG_Outcome']
project_df=project_df.assign(count=1)
project_df

,country,IEG_Outcome,count
0,Africa,Unsatisfactory,1
1,Africa,Satisfactory,1
2,Africa,Moderately Satisfactory,1
3,Africa,Unsatisfactory,1
4,Angola,Moderately Satisfactory,1
...,...,...,...
11550,Zambia,Satisfactory,1
11551,Jordan,Moderately Satisfactory,1
11552,Jamaica,Moderately Satisfactory,1
11553,Albania,Satisfactory,1


In [44]:
project_df = project_df.groupby(['country','IEG_Outcome'])['count'].sum().reset_index()
#df.sort_values(by=['col1'])
project_df.head(20)

,country,IEG_Outcome,count
867,Zimbabwe,Unsatisfactory,4
866,Zimbabwe,Satisfactory,23
865,Zimbabwe,Not Rated,2
864,Zimbabwe,Moderately Unsatisfactory,1
863,Zimbabwe,Moderately Satisfactory,12
862,Zimbabwe,Highly Unsatisfactory,1
861,Zambia,Unsatisfactory,31
860,Zambia,Satisfactory,45
859,Zambia,Not Rated,1
858,Zambia,Moderately Unsatisfactory,11


In [56]:
project_df= project_df.sort_values(by=['count'], ascending=False)

In [57]:
fig = px.bar(project_df.set_index('country'), y="count", color="IEG_Outcome", title="World bank Project",width=2000, height=800)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [49]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [50]:
list=project_df['country'].unique()

In [51]:
def f(country):
    fig = px.bar(project_df.set_index('country').loc[country],
            x="IEG_Outcome",
            y="count",
            title= 'World Bank Project for Country:'+ country,
            text="count")
    fig.show()
    
interact(f, country=(list))

interactive(children=(Dropdown(description='country', options=('India', 'Indonesia', 'China', 'Brazil', 'Pakis…

<function __main__.f(country)>

In [53]:
data1 =project_df.set_index('country').loc['Taiwan, China']
data1

IEG_Outcome    Satisfactory
count                     8
Name: Taiwan, China, dtype: object